In [19]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import mediapipe as mp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Flatten, Conv2D, MaxPooling2D, Dropout


In [20]:
mp_holistic = mp.solutions.holistic #holistic model
mp_drawing = mp.solutions.drawing_utils # drawing utilities

In [21]:
def draw_styled_landmark(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), #dot color
                mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)) # outline Color
    
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(0,255,255), thickness=2, circle_radius=2))
     
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2))
    
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

In [22]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False # Image is no longer writeable
    results = model.process(image) #make prediction
    image.flags.writeable = True # image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [23]:
def extract_keypoints(results):
    # if didnt detect any face create np zeros 33*4 , 33 because pose landmark is 33 and times 4 because it has 4 variable, X, Y, Z and Visibility the other dont have Visibility
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    # if didnt detect any face create np zeros 468*3 , 468 because face landmark is 21
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    # if didnt detect any (left/right) hand create np zeros 21*3 , 21 because hands landmark is 21
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,face, lh, rh])


In [24]:
actions = np.array(['hello', 'thanks','good','f##kyou','seeyoulater', 'ithink', 'youarepretty', 'iloveyou', 'cute'])
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
#False because next layer is Dense layer
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics='categorical_accuracy')

In [25]:
model.load_weights("action.h5")

In [26]:
colors = [(245,117,16), (102,0,204), (117,245,16), (16,117,245), (255,153,51), (204,0,204), (204,5,10)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,50+num*20), (int(prob*50), 70+num*20), colors[num%len(colors)],-1)
        # cv2.putText(output_frame, action[num], (0, 85+num*40),  cv2.FONT_HERSHEY_SIMPLEX,1, (255,255,255), 2 , cv2.LINE_AA)
        cv2.putText(output_frame, actions[num], (0, 65+num*20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2, cv2.LINE_AA)
        
    return output_frame
    

In [27]:
# 1. new detection variable
sequence = []
sentence = []
predictions = []
threshold = 0.7

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened:
        ret, frame = cap.read()
        
        #make predictions
        image, results = mediapipe_detection(frame, holistic)
        
        draw_styled_landmark(image, results) # applying or draw landmark visual to the current image
        
        #2. prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            # print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
        
        #3. viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res):
                if res[np.argmax(res)] > threshold:
                    if len(sentence) > 0:
                        if actions[np.argmax(res)]!= sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])
        
            if len(sentence) > 3:
                sentence = sentence[-3:]
            
            #viz probabilities
            image = prob_viz(res, actions, image, colors)
        
        cv2.rectangle(image, (0,0), (640,40), (245,117,16), -1)
        cv2.putText(image, " ".join(sentence), (3,30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1 , (255,255,255), 2,
                    cv2.LINE_AA)
        
        cv2.imshow("OpenCV Feed",image)
        if cv2.waitKey(10) & 0XFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 25ms/step


In [28]:
cap.release()
cv2.destroyAllWindows()